In [2]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2.service_account import Credentials
import db_dtypes

In [3]:
#BIGQUERY_CREDENTIALS_FILE_PATH = r"C:\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\TIR_group2\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH

In [4]:
BQ_CLIENT = bigquery.Client()  


In [5]:
BQ_CLIENT.project

'tir101-group2-422603'

In [ ]:
def create_dataset(dataset_name:str , location:str="asia-east1",client: bigquery.Client = BQ_CLIENT):
    """Create a dataset"""
    porject_name = client.project
    dataset_id = f"{porject_name}.{dataset_name}"

    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location

    client.create_dataset(dataset)
    print(f"created {dataset_name} in project {porject_name} ,location:{location}")
create_dataset(dataset_name="Youbike_ODS")

In [ ]:
# def youbike_gcs_src_to_bq(client: bigquery.Client = BQ_CLIENT) -> None:
#     """create external table (GCS to BQ)"""
#     query_job = client.query(
#     """
#         CREATE OR REPLACE EXTERNAL TABLE `Youbike_ODS.youbike_src`
#         WITH PARTITION COLUMNS
#         OPTIONS (
#         format = 'CSV',
#         uris = ['gs://youbike_realtime/*.csv'],
#         hive_partition_uri_prefix = 'gs://youbike_realtime/',
#         skip_leading_rows = 1,
#         max_bad_records = 1
#         );
#     """
#     )

#     query_job.result()
#     print("youbike_gcs_src_to_bq updated.")
# youbike_gcs_src_to_bq()

In [39]:
def youbike_gcs_src_to_bq_before0504(client: bigquery.Client = BQ_CLIENT) -> None:
    """create external table (GCS to BQ) for you bike data(format for API before 0504) """
    query_job = client.query(
    """
        CREATE OR REPLACE EXTERNAL TABLE `Youbike_ODS.youbike_src_before_0504`
        (
            sno INT,
            sna STRING,
            tot INT,
            sbi INT,
            sarea STRING,
            mday TIMESTAMP,
            lat FLOAT64,
            lng FLOAT64,
            ar STRING,
            sareaen STRING,
            snaen STRING,
            aren STRING,
            bemp INT,
            act INT,
            srcUpdateTime TIMESTAMP,
            updateTime TIMESTAMP,
            infoTime TIMESTAMP,
            infoDate DATE
        )
        WITH PARTITION COLUMNS
        OPTIONS (
            format = 'CSV',
            uris = ['gs://youbike_realtime/v1_data/*.csv'],
            hive_partition_uri_prefix = 'gs://youbike_realtime/v1_data/',
            skip_leading_rows = 1,
            max_bad_records = 1
        );
    """
    )

    query_job.result()
    print("youbike_gcs_src_to_bq_before0504 created.")
youbike_gcs_src_to_bq_before0504()

youbike_gcs_src_to_bq_before0504 created.


In [52]:
def youbike_gcs_src_to_bq_after0504(client: bigquery.Client = BQ_CLIENT) -> None:
    """create external table (GCS to BQ) for you bike data(format for API after 0504) """
    query_job = client.query(
    """
        CREATE OR REPLACE EXTERNAL TABLE `Youbike_ODS.youbike_src_after0504`
        (
            sno INT,
            sna  STRING,
            sarea STRING,
            mday TIMESTAMP,
            ar STRING,
            sareaen STRING,
            snaen  STRING,
            aren STRING,
            act INT,
            srcUpdateTime TIMESTAMP,
            updateTime TIMESTAMP,
            infoTime TIMESTAMP,
            infoDate DATE,
            total INT,
            available_rent_bikes INT,
            latitude FLOAT64,
            longitude FLOAT64,
            available_return_bikes INT
        )
        WITH PARTITION COLUMNS
        OPTIONS (
            format = 'CSV',
            uris = ['gs://youbike_realtime/v2_data/*.csv'],
            hive_partition_uri_prefix = 'gs://youbike_realtime/v2_data/',
            skip_leading_rows = 1,
            max_bad_records = 1
        );
    """
    )

    query_job.result()
    print("youbike_gcs_src_to_bq_after0504 created.")
youbike_gcs_src_to_bq_after0504()

youbike_gcs_src_to_bq_after0504 created.


In [29]:
def youbike_ods_before0504_create(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.youbike_ods_before0504` AS
    SELECT
        sno AS bike_station_id,
        sna AS station_name,
        sarea AS district,
        mday AS make_date,
        ar AS address,
        sareaen AS district_eng,
        snaen AS station_name_eng,
        aren AS address_eng,
        act AS disable,
        srcUpdateTime AS source_time,
        #updateTime AS create_time,
        infoTime,
        infoDate,
        tot AS total_space,
        sbi AS aval_bike,
        lat,
        lng,
        bemp AS aval_space
    FROM `Youbike_ODS.youbike_src_before_0504`;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.youbike_ods_before0504 has been created")

youbike_ods_before0504_create()

Youbike_ODS.youbike_ods_before0504 has been created


In [28]:
def youbike_ods_after0504_create(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.youbike_ods_after0504` AS
    SELECT
        sno AS bike_station_id,
        sna AS station_name,
        sarea AS district,
        mday AS make_date,
        ar AS address,
        sareaen AS district_eng,
        snaen AS station_name_eng,
        aren AS address_eng,
        act AS disable,
        srcUpdateTime AS source_time,
        #updateTime AS create_time,
        infoTime,
        infoDate,
        total AS total_space,
        available_rent_bikes AS aval_bike,
        latitude AS lat,
        longitude AS lng,
        available_return_bikes AS aval_space
    FROM `Youbike_ODS.youbike_src_after0504`;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.youbike_ods_after0504 has been created")

youbike_ods_after0504_create()

Youbike_ODS.youbike_ods_after0504 has been created


In [30]:
def youbike_ods_create(client: bigquery.Client = BQ_CLIENT):
    """create youbike ods table(combine two table(before0504 and after 0504))"""
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.youbike_ods` AS
        (   SELECT * 
                FROM `Youbike_ODS.youbike_ods_before0504`
            UNION ALL
            SELECT * 
                FROM `Youbike_ODS.youbike_ods_after0504`
        )
        ;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.youbike_ods has been created")

youbike_ods_create()

Youbike_ODS.youbike_ods has been created


In [31]:
def youbike_ods_bike_station(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.bike_station` AS
    SELECT 
        `bike_station_id`,
        `station_name`,
        `total_space`,
        `lat`,
        `lng`,
        `district` ,
        `address`,
        `disable`,
        TIMESTAMP_ADD(CURRENT_TIMESTAMP(),INTERVAL 8 HOUR) AS `create_time`,
        `source_time` 
    FROM
        (SELECT 
            * ,
            ROW_NUMBER() OVER (PARTITION by `bike_station_id` ORDER BY `source_time` DESC) AS `row_num`
        FROM `Youbike_ODS.youbike_ods_rename`) AS t1
    WHERE t1.row_num=1
    ;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.youbike_ods_bike_station has been created")

youbike_ods_bike_station()

Youbike_ODS.youbike_ods_bike_station has been created


In [44]:
def youbike_ods_bike_realtime(client: bigquery.Client = BQ_CLIENT):
    """create or update ods_bike_realtime"""
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.bike_realtime` AS
    SELECT 
        `bike_station_id`,
        `aval_bike`,
        `aval_space`,
         TIMESTAMP_ADD(CURRENT_TIMESTAMP(),INTERVAL 8 HOUR) AS `create_time`,
        `source_time` 
    FROM `Youbike_ODS.youbike_ods`;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.bike_realtime has been created")

youbike_ods_bike_realtime()

Youbike_ODS.bike_realtime has been created


In [56]:
def youbike_update_latest_create(client: bigquery.Client = BQ_CLIENT):
    """create each station latest update time """
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.youbike_update_latest` AS
    SELECT 
        `bike_station_id`,
        `source_time` 
    FROM (
        SELECT `bike_station_id`,`source_time`,
            ROW_NUMBER() OVER (PARTITION BY `bike_station_id` ORDER BY `source_time` DESC ) AS `row_num`
        FROM `Youbike_ODS.youbike_ods`
    ) AS t
    WHERE t.row_num=1;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.youbike_update_latest has been created")

youbike_update_latest_create()

Youbike_ODS.youbike_update_latest has been created


In [61]:
def youbike_create_external_new_data(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.external_new_data` AS
    SELECT
        youbike_src_after0504.sno AS bike_station_id,
        youbike_src_after0504.sna AS station_name,
        youbike_src_after0504.sarea AS district,
        youbike_src_after0504.mday AS make_date,
        youbike_src_after0504.ar AS address,
        youbike_src_after0504.sareaen AS district_eng,
        youbike_src_after0504.snaen AS station_name_eng,
        youbike_src_after0504.aren AS address_eng,
        youbike_src_after0504.act AS disable,
        youbike_src_after0504.srcUpdateTime AS source_time,
        # youbike_src_after0504.updateTime AS create_time,
        youbike_src_after0504.infoTime,
        youbike_src_after0504.infoDate,
        youbike_src_after0504.total AS total_space,
        youbike_src_after0504.available_rent_bikes AS aval_bike,
        youbike_src_after0504.latitude AS lat,
        youbike_src_after0504.longitude AS lng,
        youbike_src_after0504.available_return_bikes AS aval_space
    FROM `Youbike_ODS.youbike_src_after0504`  AS youbike_src_after0504
    INNER JOIN 
        `Youbike_ODS.youbike_update_latest` AS youbike_update_latest
    ON youbike_src_after0504.sno = youbike_update_latest.bike_station_id AND youbike_src_after0504.srcUpdateTime>youbike_update_latest.source_time
    ;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.youbike_ods_create_external_new_data has been created")

youbike_create_external_new_data()

Youbike_ODS.youbike_ods_create_external_new_data has been created


In [32]:
def create_test1(client:bigquery.Client=BQ_CLIENT):
    query_job = client.query(
        """CREATE OR REPLACE TABLE `Youbike_ODS.test1`(
            id INT,
            tv STRING,
            m_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
            );
        """
    )
    query_job.result()
    print("test1 create")
create_test1()

test1 create


In [33]:
def create_test2(client:bigquery.Client=BQ_CLIENT):
    query_job = client.query(
        """CREATE OR REPLACE TABLE `Youbike_ODS.test2`(
            id INT,
            tv STRING,
            m_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
            );
        """
    )
    query_job.result()
    print("test2 create")
create_test2()

test2 create


In [27]:
import time



123


In [34]:
def insert_test1_and_test2(client:bigquery.Client=BQ_CLIENT):
    query_job = client.query(
        """
        INSERT INTO `Youbike_ODS.test2`
        (id,tv)
        VALUES
        (1,"bt1"),
        (2,"bt2"),
        (3,"bt3")
        ;
        """
    )
    time.sleep(2)
    query_job.result()
    query_job = client.query(
        """
        INSERT INTO `Youbike_ODS.test1`
        (id,tv)
        VALUES
        (1,"a"),
        (2,"b")
        ;
        """
    )
    query_job.result()
    time.sleep(2)
    query_job = client.query(
        """
        INSERT INTO `Youbike_ODS.test2`
        (id,tv)
        VALUES
        (1,"af1"),
        (2,"af2"),
        (7,"af7")
        ;
        """
    )
    query_job.result()

    print("insert_test1_and_test2")
insert_test1_and_test2()

insert_test1_and_test2


In [ ]:
def insert_test2(client:bigquery.Client=BQ_CLIENT):
    query_job = client.query(
        """
        INSERT INTO `Youbike_ODS.test2`
        (id,tv)
        VALUES
        (1,"bt"),
        (2,"abt")
        ;
        """
    )
    query_job.result()
    print("test2 insert")
insert_test2()

test2 insert


In [31]:
def insert_test1(client:bigquery.Client=BQ_CLIENT):
    query_job = client.query(
        """
        INSERT INTO `Youbike_ODS.test1`
        (id,tv)
        VALUES
        (1,"a"),
        (2,"b")
        ;
        """
    )
    query_job.result()
    print("test1 insert")
insert_test1()

test1 insert


In [37]:
def merge_test1(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
        """
        MERGE INTO `Youbike_ODS.test1` AS target
        USING `Youbike_ODS.test2` AS source
        ON target.id = source.id AND target.m_time >= source.m_time
        WHEN NOT MATCHED BY TARGET THEN
            INSERT (id, tv, m_time)
            VALUES (source.id, source.tv, source.m_time)
        """
    )
    query_job.result()
    print("Merge operation completed successfully")

merge_test1()

Merge operation completed successfully


In [17]:
def update_data_insert_merge_into_ods(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
    """
    MERGE INTO `Youbike_ODS.youbike_ods` AS target
    USING `Youbike_ODS.youbike_src_after0504` AS source
    ON target.bike_station_id = source.sno AND target.source_time > source.srcUpdateTime
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (
            bike_station_id,
            station_name, 
            district,
            make_date,
            address,
            district_eng,
            station_name_eng,
            address_eng,
            disable,
            source_time,
            infoTime,
            infoDate,
            total_space,
            aval_bike,
            lat,
            lng,
            aval_space
            )
        VALUES (
            source.sno ,
            source.sna  ,
            source.sarea  ,
            source.mday  ,
            source.ar  ,
            source.sareaen  ,
            source.snaen  ,
            source.aren  ,
            source.act  ,
            source.srcUpdateTime  ,
            source.infoTime,
            source.infoDate,
            source.total  ,
            source.available_rent_bikes ,
            source.latitude  ,
            source.longitude  ,
            source.available_return_bikes  
        );
    """
    )
    result = query_job.result()
    return(result)

res = update_data_insert_merge_into_ods()
res

In [18]:
def update_data_insert_merge_into_ods_bike_realtime(client: bigquery.Client = BQ_CLIENT):
    """create update_data_insert_merge_into_ods_bike_realtime"""
    query_job = client.query(
    """
    MERGE INTO `Youbike_ODS.bike_realtime` AS target
    USING `Youbike_ODS.youbike_ods` AS source
    ON target.bike_station_id = source.bike_station_id AND target.source_time > source.source_time
    WHEN NOT MATCHED BY TARGET THEN
        INSERT (
            bike_station_id,
            aval_bike,
            aval_space,
            create_time,
            source_time
        )
        VALUES(
            source.bike_station_id,
            source.aval_bike,
            source.aval_space,
            CURRENT_TIMESTAMP(),
            source.source_time
        );
    """
    )
    query_job.result()
    print(f"bike_realtime has been update")

update_data_insert_merge_into_ods_bike_realtime()

bike_realtime has been update


In [39]:
def youbike_ods_fomat_time_Test(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
    """
    CREATE OR REPLACE TABLE `Youbike_ODS.ods_fomat_time_Test` AS
    SELECT *,
        FORMAT_TIMESTAMP('%F %T', TIMESTAMP_SUB(source_time, INTERVAL 8 HOUR), 'Asia/Taipei') AS new_source_time
    FROM `Youbike_ODS.youbike_ods`;
    """
    )
    query_job.result()
    print(f"Youbike_ODS.ods_fomat_time_Test has been created")

youbike_ods_fomat_time_Test()


Youbike_ODS.ods_fomat_time_Test has been created


In [45]:
def query_stackoverflow(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
        """
        SELECT *
        FROM `Youbike_ODS.bike_realtime`
        ORDER BY `create_time` DESC
        LIMIT 2000
        """
    )
    return query_job.to_dataframe()  # Convert result to DataFrame

df = query_stackoverflow()
df

,bike_station_id,aval_bike,aval_space,create_time,source_time
0,500108032,12,0,2024-05-14 16:07:11.366586+00:00,2024-05-10 17:28:28+00:00
1,500108032,12,0,2024-05-14 16:07:11.366586+00:00,2024-04-18 17:18:27+00:00
2,500108032,12,0,2024-05-14 16:07:11.366586+00:00,2024-04-19 17:08:26+00:00
3,500108032,12,0,2024-05-14 16:07:11.366586+00:00,2024-04-27 00:39:23+00:00
4,500109056,13,0,2024-05-14 16:07:11.366586+00:00,2024-05-02 18:08:26+00:00
...,...,...,...,...,...
1995,500110080,11,0,2024-05-14 16:07:11.366586+00:00,2024-05-06 21:08:25+00:00
1996,500109056,13,0,2024-05-14 16:07:11.366586+00:00,2024-05-08 03:28:22+00:00
1997,500108032,9,0,2024-05-14 16:07:11.366586+00:00,2024-04-23 22:59:23+00:00
1998,500108032,11,0,2024-05-14 16:07:11.366586+00:00,2024-05-12 10:38:25+00:00


In [ ]:
# def youbike_new_data_insert_to_ods(client: bigquery.Client = BQ_CLIENT):
#     query_job = client.query(
#     """
#     CREATE OR REPLACE TABLE `Youbike_ODS.external_new_data` AS
#     SELECT
#         youbike_src_after0504.sno AS bike_station_id,
#         youbike_src_after0504.sna AS station_name,
#         youbike_src_after0504.sarea AS district,
#         youbike_src_after0504.mday AS make_date,
#         youbike_src_after0504.ar AS address,
#         youbike_src_after0504.sareaen AS district_eng,
#         youbike_src_after0504.snaen AS station_name_eng,
#         youbike_src_after0504.aren AS address_eng,
#         youbike_src_after0504.act AS disable,
#         youbike_src_after0504.srcUpdateTime AS source_time,
#         # youbike_src_after0504.updateTime AS create_time,
#         youbike_src_after0504.infoTime,
#         youbike_src_after0504.infoDate,
#         youbike_src_after0504.total AS total_space,
#         youbike_src_after0504.available_rent_bikes AS aval_bike,
#         youbike_src_after0504.latitude AS lat,
#         youbike_src_after0504.longitude AS lng,
#         youbike_src_after0504.available_return_bikes AS aval_space
#     FROM `Youbike_ODS.youbike_src_after0504`  AS youbike_src_after0504
#     INNER JOIN 
#         `Youbike_ODS.youbike_update_latest` AS youbike_update_latest
#     ON youbike_src_after0504.sno = youbike_update_latest.bike_station_id AND youbike_src_after0504.srcUpdateTime>youbike_update_latest.source_time
#     ;
#     """
#     )
#     query_job.result()
#     print(f"Youbike_ODS.youbike_ods_create_external_new_data has been created")

# youbike_new_data_insert_to_ods()

Youbike_ODS.youbike_ods_create_external_new_data has been created


In [19]:
def query_stackoverflow(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
        """
        SELECT *
        FROM `Youbike_ODS.bike_realtime`
        ORDER BY `source_time` DESC
        LIMIT 100
        """
    )
    return query_job.to_dataframe()  # Convert result to DataFrame

df = query_stackoverflow()

In [23]:
df

,bike_station_id,aval_bike,aval_space,create_time,source_time
0,500107099,9,1,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
1,500107132,6,1,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
2,500101092,13,0,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
3,500107069,15,2,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
4,500108015,14,1,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
...,...,...,...,...,...
95,500119008,10,0,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
96,500110043,14,1,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
97,500109080,14,1,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00
98,500112066,54,0,2024-05-14 07:11:09.022493+00:00,2024-05-14 15:08:25+00:00


In [21]:
def query_stackoverflow(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
        """
        SELECT *
        FROM `Youbike_ODS.youbike_ods`
        LIMIT 2
        """
    )
    return query_job.to_dataframe()  # Convert result to DataFrame

df2 = query_stackoverflow()


In [22]:
df2

,bike_station_id,station_name,district,make_date,address,district_eng,station_name_eng,address_eng,disable,source_time,infoTime,infoDate,total_space,aval_bike,lat,lng,aval_space
0,500107018,YouBike2.0_一江公園,中山區,2024-04-21 12:06:19+00:00,一江街/松江路132巷口(西北側),Zhongshan Dist,YouBike2.0_Yijiang Park,Yijiang St. & Ln. 132， Songjiang Rd. Intersect...,1,2024-04-21 12:08:24+00:00,2024-04-21 12:06:19+00:00,2024-04-21,37,6,25.05316,121.53146,31
1,500107018,YouBike2.0_一江公園,中山區,2024-05-12 19:22:21+00:00,一江街/松江路132巷口(西北側),Zhongshan Dist,YouBike2.0_Yijiang Park,Yijiang St. & Ln. 132， Songjiang Rd. Intersect...,1,2024-05-12 23:48:22+00:00,2024-05-12 19:22:21+00:00,2024-05-12,37,17,25.05316,121.53146,20


In [ ]:
# def youbike_gcs_src_to_bq(client: bigquery.Client = BQ_CLIENT) -> None:
#     query_job = client.query(
#     """
#         CREATE OR REPLACE EXTERNAL TABLE `Youbike_ODS.youbike_src`
#         (
#             sno INT,
#             sna  STRING,
#             sarea STRING,
#             mday TIMESTAMP,
#             ar STRING,
#             sareaen STRING,
#             snaen  STRING,
#             aren STRING,
#             act INT,
#             srcUpdateTime TIMESTAMP,
#             updateTime TIMESTAMP,
#             infoTime TIMESTAMP,
#             infoDate DATE,
#             total INT,
#             available_rent_bikes INT,
#             latitude FLOAT64,
#             longitude FLOAT64,
#             available_return_bikes INT
#         )
#         WITH PARTITION COLUMNS
#         OPTIONS (
#             format = 'CSV',
#             uris = ['gs://youbike_realtime/v2_data/*.csv'],
#             hive_partition_uri_prefix = 'gs://youbike_realtime/v2_data/',
#             skip_leading_rows = 1,
#             max_bad_records = 1
#         );
#     """
#     )

#     query_job.result()
#     print("youbike_gcs_src_to_bq updated.")
# youbike_gcs_src_to_bq()

youbike_gcs_src_to_bq updated.


In [ ]:
# # ALTER TABLE mydataset.mytable
# #   RENAME COLUMN old_name TO new_name;
# CREATE TABLE `bike_station` (
#   `bike_station_id` int PRIMARY KEY,
#   `station_name` varchar(100),
#   `total_space` int,
#   `lat` decimal(8,6),
#   `lng` decimal(9,6),
#   `city_code` char(3),
#   `district` varchar(20),
#   `address` varchar(100),
#   `disable` boolean,
#   `create_time` timestamp default current_timestamp,
#   `update_time` timestamp default current_timestamp
# );

# CREATE TABLE `bike_latest_update` (
#   `bike_station_id` int PRIMARY KEY,
#   `aval_bike` int,
#   `aval_space` int,
#   `source_time` timestamp,
#   `create_time` timestamp default current_timestamp
# );

# CREATE TABLE `bike_realtime` (
#   `bike_station_id` int,
#   `aval_bike` int,
#   `aval_space` int,
#   `source_time` timestamp,
#   `create_time` timestamp default current_timestamp,
#   PRIMARY KEY (`bike_station_id`, `create_time`)
# );

In [ ]:
# def youbike_src_rename(client: bigquery.Client = BQ_CLIENT):
#     query_job = client.query(
#     """
#         ALTER TABLE `Youbike_ODS.youbike_src`
#             RENAME COLUMN `sno` TO `bike_station_id`,
#             RENAME COLUMN `sna` TO `station_name`,
#             RENAME COLUMN `sarea` TO `district`,
#             RENAME COLUMN `mday` TO `make_date`,
#             RENAME COLUMN `ar` TO `address`,
#             RENAME COLUMN `sareaen` TO `district_eng`,
#             RENAME COLUMN `snaen`  TO `station_name_eng`,
#             RENAME COLUMN `aren` TO `address_eng`,
#             RENAME COLUMN `act` TO `disable`,
#             RENAME COLUMN `srcUpdateTime` TO `source_time`,
#             RENAME COLUMN `updateTime` TO `update_time`,
#             #RENAME COLUMN `infoTime` TO `info_Time`,
#             #RENAME COLUMN `infoDate` TO `info_Date`,
#             RENAME COLUMN `total` TO `total_space`,
#             RENAME COLUMN `available_rent_bikes` TO `aval_space`,
#             RENAME COLUMN `latitude` TO `lat`,
#             RENAME COLUMN `longitude` TO `lng`,
#             RENAME COLUMN `available_return_bikes` TO `aval_return`;
#     """
#     )
#     query_job.result()
#     print(f"Youbike_ODS.youbike_src has been renamed")

# youbike_src_rename()

In [ ]:
# sno                int64
# sna               object
# tot                int64
# sbi                int64
# sarea             object
# mday              object
# lat              float64
# lng              float64
# ar                object
# sareaen           object
# snaen             object
# aren              object
# bemp               int64
# act                int64
# srcUpdateTime     object
# updateTime        object
# infoTime          object
# infoDate          object

In [ ]:
# def youbike_bqsrc_to_ods_station_name(client:bigquery.Client=BQ_CLIENT):
#     query_job=client.query(
#        """"
#        CREATE OR REPLACE EXTERNAL TABLE `Youbike_ODS.bike_station` AS 
#        SELECT
#          CAST(`sno` AS INT) AS  `bike_station_id`,
#          CAST(`sna` AS STRING) AS  `station_name`,
#          CAST(`sarea` AS INT) AS  `total_space`,
#          CAST(`sarea` AS STRING) AS  `district`,
#          CAST(`sarea` AS FLOAT64) AS  `lat`,
#          CAST(`sarea` AS FLOAT64) AS  `lng`,
#          CAST(`sarea` AS STRING) AS  `address`,
#          CAST(`sarea` AS INT) AS  `disable`
#         FROM 'Youbike_ODS.youbike_src'
       
#        """ 
#     ) 

In [ ]:
# CREATE TABLE `bike_station` (
#   `bike_station_id` int PRIMARY KEY,
#   `station_name` varchar(100),
#   `total_space` int,
#   `lat` decimal(8,6),
#   `lng` decimal(9,6),
#   `city_code` char(3),
#   `district` varchar(20),
#   `address` varchar(100),
#   `disable` boolean,
#   `create_time` timestamp,
#   `update_time` timestamp
# );

In [22]:
def delete_dataset(dataset_name:str ,client: bigquery.Client = BQ_CLIENT):  #刪除dataset名稱
    porject_name = client.project
    dataset_id = f"{porject_name}.{dataset_name}"
    client.delete_dataset(dataset=dataset_id,delete_contents=True,not_found_ok=True)
    print(f"dataset {dataset_name} in project {porject_name} has been delete")
delete_dataset("Youbike_ODS")

dataset Youbike_ODS in project tir101-group2-422603 has been delete


In [7]:
# def etl_sales_src_to_bq(client: bigquery.Client = BQ_CLIENT) -> None:
#     query_job = client.query(
#         """
#         CREATE OR REPLACE EXTERNAL TABLE `bike_v1.youbike_ods_test1`
#         WITH PARTITION COLUMNS
#         OPTIONS (
#         format = 'CSV',
#         uris = ['gs://youbike_realtime/*.csv'],
#         hive_partition_uri_prefix = 'gs://youbike_realtime/',
#         skip_leading_rows = 1,
#         max_bad_records = 1
#         );
#         """
#     )

#     query_job.result()

#     print("youbike updated.")

In [5]:
etl_sales_src_to_bq()

youbike updated.


In [43]:
def query_stackoverflow(client: bigquery.Client = BQ_CLIENT):
    query_job = client.query(
        """
        SELECT *
        FROM `Youbike_ODS.youbike_src`
        LIMIT 3
        """
    )
    return query_job.to_dataframe()  # Convert result to DataFrame

df = query_stackoverflow()


In [67]:
df=pd.read_csv("dt=04_24_2024_01_50.csv")
df.dtypes

sno                int64
sna               object
tot                int64
sbi                int64
sarea             object
mday              object
lat              float64
lng              float64
ar                object
sareaen           object
snaen             object
aren              object
bemp               int64
act                int64
srcUpdateTime     object
updateTime        object
infoTime          object
infoDate          object
dtype: object

In [50]:
df = pd.read_csv("youbiketest.csv")
df.head(2)

,sno,sna,sarea,mday,ar,sareaen,snaen,aren,act,srcUpdateTime,updateTime,infoTime,infoDate,total,available_rent_bikes,latitude,longitude,available_return_bikes,dt
0,500101005,YouBike2.0_辛亥復興路口西北側,大安區,2024-05-06 07:58:18+00:00,復興南路二段368號,Daan Dist.,YouBike2.0_Xinhai Fuxing Rd. Intersection (Nor...,No. 368， Sec. 2， Fuxing S. Rd.,1,2024-05-06 07:58:26+00:00,2024-05-06 07:58:51+00:00,2024-05-06 07:58:18+00:00,2024-05-06,16,8,25.02153,121.54299,8,05_06_2024
1,500101006,YouBike2.0_復興南路二段280號前,大安區,2024-05-06 07:57:36+00:00,復興南路二段280號,Daan Dist.,YouBike2.0_No. 280， Sec. 2， Fuxing S. Rd.,No. 280， Sec. 2， Fuxing S. Rd.,1,2024-05-06 07:58:26+00:00,2024-05-06 07:58:51+00:00,2024-05-06 07:57:36+00:00,2024-05-06,11,1,25.02429,121.54328,10,05_06_2024


In [35]:
df.to_csv("youbiketest.csv",index=False,encoding="utf-8")

In [37]:
df

,sno,sna,sarea,mday,ar,sareaen,snaen,aren,act,srcUpdateTime,updateTime,infoTime,infoDate,total,available_rent_bikes,latitude,longitude,available_return_bikes,dt
0,500101005,YouBike2.0_辛亥復興路口西北側,大安區,2024-05-06 07:58:18+00:00,復興南路二段368號,Daan Dist.,YouBike2.0_Xinhai Fuxing Rd. Intersection (Nor...,No. 368， Sec. 2， Fuxing S. Rd.,1,2024-05-06 07:58:26+00:00,2024-05-06 07:58:51+00:00,2024-05-06 07:58:18+00:00,2024-05-06,16,8,25.021530,121.542990,8,05_06_2024
1,500101006,YouBike2.0_復興南路二段280號前,大安區,2024-05-06 07:57:36+00:00,復興南路二段280號,Daan Dist.,YouBike2.0_No. 280， Sec. 2， Fuxing S. Rd.,No. 280， Sec. 2， Fuxing S. Rd.,1,2024-05-06 07:58:26+00:00,2024-05-06 07:58:51+00:00,2024-05-06 07:57:36+00:00,2024-05-06,11,1,25.024290,121.543280,10,05_06_2024
2,500101009,YouBike2.0_新生南路三段66號前,大安區,2024-05-06 07:24:18+00:00,新生南路三段66號東側,Daan Dist.,YouBike2.0_No. 66， Sec. 3， Xinsheng S. Rd.,No. 66， Sec. 3， Xinsheng S. Rd. (East),1,2024-05-06 07:58:26+00:00,2024-05-06 07:58:51+00:00,2024-05-06 07:24:18+00:00,2024-05-06,16,1,25.019760,121.533840,15,05_06_2024
3,500101019,YouBike2.0_和平臥龍街口,大安區,2024-05-06 07:58:19+00:00,臥龍街1號,Daan Dist.,YouBike2.0_Heping Wolong St. Intersection,No. 1， Wolong St.,1,2024-05-06 07:58:26+00:00,2024-05-06 07:58:51+00:00,2024-05-06 07:58:19+00:00,2024-05-06,15,9,25.024461,121.546705,6,05_06_2024
4,500101034,YouBike2.0_臺大動物醫院前,大安區,2024-05-06 07:53:14+00:00,基隆路三段155巷/基隆路四段口,Daan Dist.,YouBike2.0_NTU Veterinary Hospital,Ln. 155， Sec. 3， Keelung Rd. Intersection,1,2024-05-06 07:58:26+00:00,2024-05-06 07:58:51+00:00,2024-05-06 07:53:14+00:00,2024-05-06,18,17,25.015760,121.542910,0,05_06_2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340932,500119071,YouBike2.0_臺大計資中心南側,臺大公館校區,2024-05-10 10:08:20+00:00,臺大計算機網路資訊中心南側,NTU Dist,YouBike2.0_NTU Computer and Information Networ...,NTU Computer and Information Networking Center...,1,2024-05-10 10:08:25+00:00,2024-05-10 10:08:51+00:00,2024-05-10 10:08:20+00:00,2024-05-10,34,15,25.020360,121.539920,4,05_10_2024
1340933,500119072,YouBike2.0_臺大原分所北側,臺大公館校區,2024-05-10 10:08:20+00:00,臺大原子與分子科學研究所東北側,NTU Dist,YouBike2.0_NTU Institute of Atomic and Molecul...,NTU Institute of Atomic and Molecular Sciences...,1,2024-05-10 10:08:25+00:00,2024-05-10 10:08:51+00:00,2024-05-10 10:08:20+00:00,2024-05-10,12,11,25.018810,121.538680,1,05_10_2024
1340934,500119075,YouBike2.0_臺大第一活動中心西南側,臺大公館校區,2024-05-10 10:07:19+00:00,第一學生活動中心東南側,NTU Dist,YouBike2.0_NTU 1st Student Activity Center(Sou...,NTU 1st Student Activity Center(Southeast),1,2024-05-10 10:08:25+00:00,2024-05-10 10:08:51+00:00,2024-05-10 10:07:19+00:00,2024-05-10,40,17,25.017610,121.539950,3,05_10_2024
1340935,500119081,YouBike2.0_臺大教研館北側,臺大公館校區,2024-05-10 10:07:19+00:00,臺大雅頌廣場東南側,NTU Dist,YouBike2.0_NTU Teaching and Research Building(...,NTU The Odeum Square(Southeast),1,2024-05-10 10:08:25+00:00,2024-05-10 10:08:51+00:00,2024-05-10 10:07:19+00:00,2024-05-10,11,5,25.013370,121.539090,6,05_10_2024


In [56]:
from google.cloud import storage
storage_Client = storage.Client()

In [57]:
bucket = storage_Client.bucket("youbike_realtime")

In [65]:
file = bucket.get_blob("v2_data/dt=05_14_2024/13_50.csv")

In [66]:
file.download_to_filename("13_50.csv")